In [5]:
import sys
import time
import os
import webbrowser
import subprocess
import urllib.parse

import pyttsx3
import speech_recognition as sr
# import pywhatkit  # optional — not used in this example
# import pyautogui  # only required if you want GUI automation fallback

# ---------- Setup ----------
speaker = pyttsx3.init()
mic = sr.Recognizer()

# Helper: open a URL in a cross-platform way
def open_url_cross_platform(url: str):
    try:
        webbrowser.open(url)                     # standard way (works in most cases)
    except Exception:
        if sys.platform.startswith("darwin"):    # macOS
            subprocess.call(["open", url])
        elif sys.platform.startswith("win"):     # Windows
            os.startfile(url)
        else:                                    # Linux / others
            subprocess.call(["xdg-open", url])

# Helper: normalize phone numbers for whatsapp://send (strip +, spaces)
def normalize_phone_for_whatsapp(phone: str) -> str:
    # Remove non-digit characters
    return "".join(ch for ch in phone if ch.isdigit())

# ---------- Your app ----------
speaker.say("Hello, I am Jarvis")
speaker.runAndWait()

with sr.Microphone() as source:
    mic.adjust_for_ambient_noise(source, duration=0.8)  # helps recognition
    print("Listening...")
    voice = mic.listen(source)
    command = ""
    try:
        text = mic.recognize_google(voice)
        print("Heard:", text)
        command = text.lower()

        if "open youtube" in command:
            open_url_cross_platform("https://www.youtube.com")
        elif "open google" in command:
            open_url_cross_platform("https://www.google.com")
        elif "open whatsapp" in command:
            # Try to open desktop app first (mac path example) — fallback to web
            if sys.platform.startswith("darwin"):
                # macOS WhatsApp app path (may vary); fallback to web if fails
                app_path = "/Applications/WhatsApp.app"
                if os.path.exists(app_path):
                    subprocess.call(["open", app_path])
                else:
                    open_url_cross_platform("https://web.whatsapp.com")
            elif sys.platform.startswith("win"):
                # On Windows try to launch WhatsApp if installed; else open web
                try:
                    subprocess.Popen(["C:\\Users\\%USERNAME%\\AppData\\Local\\WhatsApp\\WhatsApp.exe"])
                except Exception:
                    open_url_cross_platform("https://web.whatsapp.com")
            else:
                open_url_cross_platform("https://web.whatsapp.com")

        elif "open instagram" in command:
            open_url_cross_platform("https://www.instagram.com")
        elif "open facebook" in command:
            open_url_cross_platform("https://www.facebook.com")
        elif "open twitter" in command:
            open_url_cross_platform("https://www.twitter.com")
        elif "open linkedin" in command:
            open_url_cross_platform("https://www.linkedin.com")
        elif "open github" in command:
            open_url_cross_platform("https://www.github.com")
        elif "open stackoverflow" in command:
            open_url_cross_platform("https://stackoverflow.com")

        elif "send message" in command:
            try:
                import pyautogui  # optional: used for pressing Enter as a final step
            except Exception:
                pyautogui = None

            # Define contacts dictionary - USER MUST UPDATE THIS to real numbers
            # Store numbers in international format (can include + or spaces; we'll normalize)
            contacts = {
                "friend": "+91 78782 75650",
                # add more: "sister": "+91xxxxxxxxxx"
            }

            speaker.say("What message should I send?")
            speaker.runAndWait()

            with sr.Microphone() as source:
                mic.adjust_for_ambient_noise(source, duration=0.6)
                print("Listening for message...")
                voice_msg = mic.listen(source)
                message = mic.recognize_google(voice_msg)
                print(f"Message: {message}")

            speaker.say("Who should I send it to?")
            speaker.runAndWait()

            with sr.Microphone() as source:
                mic.adjust_for_ambient_noise(source, duration=0.6)
                print("Listening for recipient...")
                voice_recipient = mic.listen(source)
                recipient_name = mic.recognize_google(voice_recipient).lower()
                print(f"Recipient: {recipient_name}")

            if recipient_name in contacts:
                raw_number = contacts[recipient_name]
                phone_number = normalize_phone_for_whatsapp(raw_number)
                # URL-encode message
                encoded_msg = urllib.parse.quote(message)
                # Use whatsapp:// URL scheme (mobile/desktop app), fallback to web.whatsapp
                url = f"whatsapp://send?phone={phone_number}&text={encoded_msg}"
                try:
                    open_url_cross_platform(url)
                    # Small wait for app/web to open
                    time.sleep(3)
                    # If using web.whatsapp, the user may need to press Enter or click send.
                    # If pyautogui is available and you know the page has focus, send Enter:
                    if pyautogui:
                        pyautogui.press("enter")
                    speaker.say(f"Message sent to {recipient_name}")
                    speaker.runAndWait()
                except Exception as e:
                    print("Primary WhatsApp URL open failed:", e)
                    # fallback: open web whatsapp with prefilled text via web.whatsapp link (works for web)
                    web_url = f"https://web.whatsapp.com/send?phone={phone_number}&text={encoded_msg}"
                    open_url_cross_platform(web_url)
                    speaker.say("Opened WhatsApp web. Please finish sending manually.")
                    speaker.runAndWait()
            else:
                speaker.say("Contact not found.")
                speaker.runAndWait()

    except Exception as e:
        print("Error recognizing or executing command:", str(e))
        speaker.say("Sorry, I couldn't understand. Please try again.")
        speaker.runAndWait()


Listening...
Heard: parent function
